In [ ]:
#This is the code to create a Lomb-Scargle Periodogram for a given light curve
import numpy as np
from astropy.timeseries import LombScargle

def calculate_lomb_scargle_period(txt_file, delimiter=r"\s+"):
    # Read the .txt file into a DataFrame
    try:
        df = pd.read_csv(txt_file, delimiter=delimiter, skiprows=2, header=None, usecols=[1,4], names=['Time', 'Magnitude'])
    except pd.errors.EmptyDataError:
        print(f"The file '{txt_file}' is empty.")
        return None
    except pd.errors.ParserError:
        print(f"Error parsing '{txt_file}'. Please check the file format.")
        return None

    # Compute Lomb-Scargle periodogram
    frequency, power = LombScargle(df['Time'], df['Magnitude']).autopower()
    
    # Estimate periods from the periodogram
    significance_threshold = 0.5  # Change this threshold as needed
    periods = 1 / frequency[power > significance_threshold * np.max(power)]
    
    return periods, frequency, power

# Example usage:
txt_file = r"C:\Users\nic51\TESS RESEARCH\Extracted_tarballs\sector27\cam4_ccd1\lc_discovery\lc_2020ork_cleaned.txt"
delimiter = r"\s+"  # Change this to the appropriate delimiter (e.g., ',', ' ', etc.)
periods, frequency, power = calculate_lomb_scargle_period(txt_file, delimiter)
print("Periods:", periods)
print("Frequency:", frequency)
print("Power:", power)


In [ ]:
#This code is used to perform a fourier transform on the light curve
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

def compute_fourier_transform(df):
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(df['Time'], df['Magnitude'])
    line = slope * df['Time'] + intercept
    
    # Remove the mean magnitude to eliminate the DC component
    magnitude_mean = np.mean(df['Magnitude'])
    df['Magnitude'] -= magnitude_mean
    
    # Compute the Fourier Transform of the magnitude
    mag_fft = np.fft.fft(df['Magnitude'])
    freqs = np.fft.fftfreq(len(df['Time']), df['Time'][1] - df['Time'][0])
    
    return freqs, np.abs(mag_fft)

def plot_light_curve_with_fft(txt_file, delimiter=r"\s+", title="Light Curve"):
    # Read the .txt file into a DataFrame
    try:
        df = pd.read_csv(txt_file, delimiter=delimiter, skiprows=2, header=None, usecols=[1,4], names=['Time', 'Magnitude'])
    except pd.errors.EmptyDataError:
        print(f"The file '{txt_file}' is empty.")
        return
    except pd.errors.ParserError:
        print(f"Error parsing '{txt_file}'. Please check the file format.")
        return
    
    freqs, mag_fft = compute_fourier_transform(df)
    
    # Plot the light curve and its Fourier Transform
    plt.figure(figsize=(12, 6))
    
    plt.subplot(2, 1, 1)
    plt.plot(df['Time'], df['Magnitude'], marker='o', linestyle='-', color='blue', label='Light Curve')
    plt.plot(df['Time'], slope * df['Time'] + intercept, color='red', label='Line of Best Fit')
    plt.xlabel('Time', fontsize=12)
    plt.ylabel('Magnitude', fontsize=12)
    plt.title(title, fontsize=16)
    plt.gca().invert_yaxis()
    plt.grid(True)
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(freqs, mag_fft, color='green')
    plt.xlabel('Frequency', fontsize=12)
    plt.ylabel('Magnitude FFT', fontsize=12)
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Example usage:
txt_file = r"C:\Users\nic51\TESS RESEARCH\Extracted_tarballs\sector25\cam4_ccd1\lc_discovery\lc_2020lkf_cleaned.txt"
delimiter = r"\s+"  # Change this to the appropriate delimiter (e.g., ',', ' ', etc.)
title = "2020lkf"  # Change this to the appropriate title
plot_light_curve_with_fft(txt_file, delimiter, title)
